# A/B Testing the Udacity Website

In these exercises, we'll be analyzing data on user behavior from an experiment run by Udacity, the online education company. More specifically, we'll be looking at a test Udacity ran to improve the onboarding process on their site.

Udacity's test is an example of an "A/B" test, in which some portion of users visiting a website (or using an app) are randomly selected to see a new version of the site. An analyst can then compare the behavior of users who see a new website design to users seeing their normal website to estimate the effect of rolling out the proposed changes to all users. While this kind of experiment has it's own name in industry (A/B testing), to be clear it's just a randomized experiment, and so everything we've learned about potential outcomes and randomized experiments apply here. 

(Udacity has generously provides the data from this test under an Apache open-source license, and you can find their [original writeup here](https://www.kaggle.com/tammyrotem/ab-tests-with-python/notebook). If you're interested in learning more on A/B testing in particular, it seems only fair while we use their data to flag they have a full course on the subject [here](https://www.udacity.com/course/ab-testing--ud257).)

## Udacity's Test

The test [is described by Udacity as follows](https://www.kaggle.com/tammyrotem/ab-tests-with-python/notebook): 

At the time of this experiment, Udacity courses currently have two options on the course overview page: "start free trial", and "access course materials".

**Current Conditions Before Change**

- If the student clicks "start free trial", they will be asked to enter their credit card information, and then they will be enrolled in a free trial for the paid version of the course. After 14 days, they will automatically be charged unless they cancel first.
- If the student clicks "access course materials", they will be able to view the videos and take the quizzes for free, but they will not receive coaching support or a verified certificate, and they will not submit their final project for feedback.

**Description of Experimented Change**

- In the experiment, Udacity tested a change where if the student clicked "start free trial", they were asked how much time they had available to devote to the course.
- If the student indicated 5 or more hours per week, they would be taken through the checkout process as usual. If they indicated fewer than 5 hours per week, a message would appear indicating that Udacity courses usually require a greater time commitment for successful completion, and suggesting that the student might like to access the course materials for free.
- At this point, the student would have the option to continue enrolling in the free trial, or access the course materials for free instead. This [screenshot](images/udacity_checkyoureready.png) shows what the experiment looks like.

**Udacity's Hope is that...**:

> this might set clearer expectations for students upfront, thus reducing the number of frustrated students who left the free trial because they didn't have enough time -- without significantly reducing the number of students to continue past the free trial and eventually complete the course. If this hypothesis held true, Udacity could improve the overall student experience and improve coaches' capacity to support students who are likely to complete the course.



## Gradescope Autograding

Please follow [all standard guidance](https://www.practicaldatascience.org/html/autograder_guidelines.html) for submitting this assignment to the Gradescope autograder, including storing your solutions in a dictionary called `results` and ensuring your notebook runs from the start to completion without any errors.

For this assignment, please name your file `exercise_abtesting.ipynb` before uploading.

You can check that you have answers for all questions in your `results` dictionary with this code:

```python
assert set(results.keys()) == {
    "ex4_avg_oec",
    "ex5_avg_guardrail",
    "ex7_ttest_pvalue",
    "ex9_ttest_pvalue_clicks",
    "ex10_num_obs",
    "ex11_guard_ate",
    "ex11_guard_pvalue",
    "ex11_oec_ate",
    "ex11_oec_pvalue",
    "ex14_se_treatment",
}
```


### Submission Limits

Please remember that you are **only allowed FOUR submissions to the autograder.** Your last submission (if you submit 4 or fewer times), or your third submission (if you submit more than 4 times) will determine your grade Submissions that error out will **not** count against this total.

That's one more than usual in case there are issues with exercise clarity.

## Import the Data

### Exercise 1

Begin by importing Udacity's data on user behavior [here.](https://github.com/nickeubank/MIDS_Data/tree/master/udacity_AB_testing) 

There are TWO datasets for this test — one for the control data (users who saw the original design), and one for treatment data (users who saw the experimental design). Udacity decided to show their test site to 1/2 of visitors, so there are roughly the same number of users appearing in each dataset (though this is not a requirement of AB tests).

Please remember to load the data directly from github to assist the autograder.

In [1]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

pd.set_option("mode.copy_on_write", True)

# load the 1st dataset
control = pd.read_csv(
    "https://github.com/nickeubank/MIDS_Data/raw/master/udacity_AB_testing/control_data.csv"
)

In [2]:
control.head()

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7723,687,134.0,70.0
1,"Sun, Oct 12",9102,779,147.0,70.0
2,"Mon, Oct 13",10511,909,167.0,95.0
3,"Tue, Oct 14",9871,836,156.0,105.0
4,"Wed, Oct 15",10014,837,163.0,64.0


In [3]:
# load the 2nd dataset
experiment = pd.read_csv(
    "https://github.com/nickeubank/MIDS_Data/raw/master/udacity_AB_testing/experiment_data.csv"
)

In [4]:
experiment.head()

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7716,686,105.0,34.0
1,"Sun, Oct 12",9288,785,116.0,91.0
2,"Mon, Oct 13",10480,884,145.0,79.0
3,"Tue, Oct 14",9867,827,138.0,92.0
4,"Wed, Oct 15",9793,832,140.0,94.0


### Exercise 2

Explore the data. Can you identify the unit of observation of the data (e.g. what is represented by each row)?

(Note this is not the only way that A/B test data can be collected and/or reported — this is just what Udacity provided, presumably to help address privacy concerns.)

In [5]:
print("\nControl Group Info:")
print(control.info())
print("\nExperiment Group Info:")
print(experiment.info())


Control Group Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         37 non-null     object 
 1   Pageviews    37 non-null     int64  
 2   Clicks       37 non-null     int64  
 3   Enrollments  23 non-null     float64
 4   Payments     23 non-null     float64
dtypes: float64(2), int64(2), object(1)
memory usage: 1.6+ KB
None

Experiment Group Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         37 non-null     object 
 1   Pageviews    37 non-null     int64  
 2   Clicks       37 non-null     int64  
 3   Enrollments  23 non-null     float64
 4   Payments     23 non-null     float64
dtypes: float64(2), int64(2), object(1)
memory usage: 1.6+ KB
None


In [6]:
print("\nControl Group Summary Statistics:")
print(control.describe())
print("\nExperiment Group Summary Statistics:")
print(experiment.describe())

# check for missing values in both datasets
print("\nMissing Values in Control Group:")
print(control.isnull().sum())
print("\nMissing Values in Experiment Group:")
print(experiment.isnull().sum())


Control Group Summary Statistics:
          Pageviews      Clicks  Enrollments    Payments
count     37.000000   37.000000    23.000000   23.000000
mean    9339.000000  766.972973   164.565217   88.391304
std      740.239563   68.286767    29.977000   20.650202
min     7434.000000  632.000000   110.000000   56.000000
25%     8896.000000  708.000000   146.500000   70.000000
50%     9420.000000  759.000000   162.000000   91.000000
75%     9871.000000  825.000000   175.000000  102.500000
max    10667.000000  909.000000   233.000000  128.000000

Experiment Group Summary Statistics:
          Pageviews      Clicks  Enrollments    Payments
count     37.000000   37.000000    23.000000   23.000000
mean    9315.135135  765.540541   148.826087   84.565217
std      708.070781   64.578374    33.234227   23.060841
min     7664.000000  642.000000    94.000000   34.000000
25%     8881.000000  722.000000   127.000000   69.000000
50%     9359.000000  770.000000   142.000000   91.000000
75%     9737.00

> The unit of observation in control and experiment datasets is daily data on webpage interactions. Each row represents aggregated data for a single day, including metrics like pageviews, clicks, enrollments, and payments. Both datasets consist of 37 entries, suggesting data was collected over 37 days.
There are 14 missing values in both the ``Enrollments`` and ``Payments`` columns for both groups, indicating these metrics were not recorded or available for all days.

## Pick your measures

### Exercise 3

The easiest way to analyze this data is to stack it into a single dataset where each observation is a day-treatment-arm (so you should end up with two rows per day, one for those who are in the treated groups, and one for those who were in the control group). Note that currently nothing in the data identifies whether a given observation is a treatment group observation or a control group observation, so you'll want to make sure to add a "treatment" indicator variable.

The variables in the data are:

- Pageviews: number of unique users visiting homepage
- Clicks: number of those users clicking "Start Free Trial"
- Enrollments: Number of people enrolling in trial
- Payments: Number of people who eventually pay for the service. Note the `payment` column reports payments for the users who first visited the site on the reported date, not payments occurring on the reported date.

In [7]:
# stack into single dataset

# add a 'Group' column to each df to indicate the group: control or treatment
control["Group"] = "Control"
experiment["Group"] = "Experiment"

# combine the two dfs into a single dataset
combined_data = pd.concat([control, experiment], axis=0)

# reorder the combined dataset if desired
combined_data = combined_data[
    ["Date", "Group", "Pageviews", "Clicks", "Enrollments", "Payments"]
]

# reset index of the combined df
combined_data.reset_index(drop=True, inplace=True)

print(combined_data.head())

          Date    Group  Pageviews  Clicks  Enrollments  Payments
0  Sat, Oct 11  Control       7723     687        134.0      70.0
1  Sun, Oct 12  Control       9102     779        147.0      70.0
2  Mon, Oct 13  Control      10511     909        167.0      95.0
3  Tue, Oct 14  Control       9871     836        156.0     105.0
4  Wed, Oct 15  Control      10014     837        163.0      64.0


### Exercise 4

Given Udacity's goals, what outcome are they hoping will be impacted by their manipulation?

Or, to ask the same question in the language of the Potential Outcomes Framework, what is their $Y$?

Or to ask the same question in the language of Kohavi, Tang and Xu, what is their *Overall Evaluation Criterion (OEC)*?

(I'm only asking one question, I'm just trying to phrase it using different terminologies we've encountered to help you see how they all fit together)

When you feel like you have your answer, please compute it. Store the average value of the variable in `results` under the key `ex4_avg_oec`. **Please round your answer to 4 decimal places.**

NOTE: You'll probably notice you have two choices to make when it comes to actually computing the OEC. 

- You could probably imagine either computing a ratio or a difference of two things — please calculate the difference.
- You may also be unsure whether to normalize by `Clicks`. Normalizing by clicks will help account for variation that comes from day-to-day variation in users, so it's a good thing to do. With infinite data, you'd expect to get the same results without normalizing by `Clicks` (since on average the same share of users are in each arm of the experiment), but for finite data it's a good strategy. Note that this is only ok because users make the choice to click or not *before* they see different versions of the website (it is "pre-treatment").

Just to make sure you're on track, your measure should have an average value of *about* 9%.

In [8]:
# filter out rows with missing 'Enrollments' or 'Payments' values
# control_clean = control.dropna(subset=["Enrollments", "Payments"])
# experiment_clean = experiment.dropna(subset=["Enrollments", "Payments"])

# calculate the payment rate per click for combined group
combined_data["payment_rate_per_click"] = (
    combined_data["Enrollments"] - combined_data["Payments"]
) / combined_data["Clicks"]

# compute the average difference in payment rate per click
avg_diff = combined_data["payment_rate_per_click"].mean()

results = {}
results["ex4_avg_oec"] = round(avg_diff, 4)

print(
    "The average difference in payment rate per click, normalized "
    f"and expressed as a percentage, is approximately: "
    f"{results['ex4_avg_oec']}%"
)

The average difference in payment rate per click, normalized and expressed as a percentage, is approximately: 0.0941%


In [10]:
results

{'ex4_avg_oec': 0.0941}

### Exercise 5

Given Udacity's goals, what outcome are they hoping will *not* be impacted by their manipulation? In other words, what do they want to measure to ensure their treatment doesn't have unintended negative consequences that might be really costly to their operation?

Note that while this isn't how Kohavi, Tang, and Xu use the term "guardrail metrics" — they usually use the term to refer to things we measure to ensure the experiment is working the way it should — some people would also use the term "guardrail metrics" for something that could be impacted even if the experiment is working correctly, but which the organization wants to track to ensure they aren't impacted because they are deemed really important.

Again, please normalize by `Clicks`. Store the average value of this guardrail metric as `ex5_avg_guardrail` and **round your answer to 4 decimal places.**

In [32]:
# Calculate Payments per Click
combined_data["Payments_per_Click"] = (
    combined_data["Payments"] / combined_data["Clicks"]
)

# Calculate the average of Payments per Click, rounded to 4 decimal places
ex5_avg_guardrail = round(combined_data["Payments_per_Click"].mean(), 4)

results["ex5_avg_guardrail"] = ex5_avg_guardrail

print(
    f"The average value of the guardrail metric (CTR), "
    f"normalized and rounded is: {ex5_avg_guardrail}"
)

The average value of the guardrail metric (CTR), normalized and rounded is: 0.1158


In [33]:
results

{'ex4_avg_oec': 0.0941,
 'ex5_avg_guardrail': 0.1158,
 'ex7_ttest_pvalue': 0.8877,
 'ex9_ttest_pvalue_clicks': 0.9264,
 'ex10_num_obs': 46,
 'ex11_oec_ate': 1.5888,
 'ex11_guard_ate': -0.0165,
 'ex11_oec_pvalue': 0.1319,
 'ex11_guard_pvalue': 0.8753,
 'ex14_se_treatment': 0.0066}

## Validating The Data

### Exercise 6

Whenever you are working with experimental data, the first thing you want to do is verify that users actually were randomly sorted into the two arms of the experiment. In this data, half of users were supposed to be shown the old version of the site and half were supposed to see the new version.

`Pageviews` tells you how many unique users visited the welcome site we are experimenting on. `Pageviews` is what is sometimes called an "invariant" or "guardrail" variable, meaning that it shouldn't vary across treatment arms—after all, people have to visit the site before they get a chance to see the treatment, so there's no way that being assigned to treatment or control should affect the number of pageviews assigned to each group.

"Invariant" variables are also an example of what are known as a "pre-treatment" variable, because pageviews are determined before users are manipulated in any way. That makes it analogous to gender or age in experiments where you have demographic data—a person's age and gender are determined before they experience any manipulations, so the value of any pre-treatment attributes should be the same across the two arms of our experiment. This is what we've previously called "checking for balance," If pre-treatment attributes aren't balanced, then we may worry our attempt to randomly assign people to different groups failed.  Kohavi, Tang and Xu call this a "trust-based guardrail metric" because it helps us determine if we should trust our data.

To test the quality of the randomization, calculate the average number of pageviews for the treated group and for the control group. Do they look similar?


In [13]:
# calculate the avg of pageviews for the control group
avg_pageviews_control = combined_data[combined_data["Group"] == "Control"][
    "Pageviews"
].mean()

# calculate the avg of pageviews for the treated (experiment) group
avg_pageviews_experiment = combined_data[combined_data["Group"] == "Experiment"][
    "Pageviews"
].mean()

print(f"Average pageviews for control group: {round(avg_pageviews_control, 4)}")
print(f"Average pageviews for experiment group: {round(avg_pageviews_experiment, 4)}")

Average pageviews for control group: 9339.0
Average pageviews for experiment group: 9315.1351


>Based on the calculation result, the average number of pageviews for the treated group and for the control group look similar.


### Exercise 7

"Similar" is a tricky concept -- obviously, we expect *some* differences across groups since users were *randomly* divided across treatment arms. The question is whether the differences between groups are larger than we'd expect to emerge given our random assignment process. To evaluate this, let's use a `ttest` to test the statistical significance of the differences we see. 

**Note**: Remember that scipy functions don't accept `pandas` objects, so you use a scipy function, you have to pass the numpy vectors underlying your data with the `.values` operator (e.g. `df.my_column.values`). 

Does the difference in `pageviews` look statistically significant?

Store the resulting p-value in `ex7_ttest_pvalue` **rounded to four decimal places.**

In [14]:
from scipy.stats import ttest_ind
import numpy as np

# extract pageviews as numpy arrays for both groups
pageviews_control = combined_data[combined_data["Group"] == "Control"][
    "Pageviews"
].values
pageviews_experiment = combined_data[combined_data["Group"] == "Experiment"][
    "Pageviews"
].values

# perform the t-test on the numpy vectors
ttest_result = ttest_ind(pageviews_control, pageviews_experiment)

# extract the p-value and round it to four decimal places
ex7_ttest_pvalue = round(ttest_result.pvalue, 4)
results["ex7_ttest_pvalue"] = ex7_ttest_pvalue

print(f"t-test p-value for Pageviews: {ex7_ttest_pvalue}")

t-test p-value for Pageviews: 0.8877


In [15]:
results

{'ex4_avg_oec': 0.0941,
 'ex5_avg_guardrail': 0.0822,
 'ex7_ttest_pvalue': 0.8877}

### Exercise 8

`Pageviews` is not the only "pre-treatment" variable in this data we can use to evaluate balance/use as a guardrail metric. What other measure is pre-treatment? Review the description of the experiment if you're not sure.

>The other pre-treatment variable in this data, alongside ``Pageviews``, is ``Clicks``. This variable represents the number of users who clicked to start a free trial, and it's considered pre-treatment because the action of clicking occurs before users are exposed to any experiment-related differences. It helps verify the experiment's randomization by showing whether user engagement before the treatment was evenly distributed across both groups.

### Exercise 9

Check if the other pre-treatment variable is also balanced. Store the p-value of your test of difference in `results` under the key `"ex9_ttest_pvalue_clicks"` **rounded to four decimal places.**

In [16]:
# same code structure with ex7
clicks_control = combined_data[combined_data["Group"] == "Control"]["Clicks"].values
clicks_experiment = combined_data[combined_data["Group"] == "Experiment"][
    "Clicks"
].values

ttest_result_clicks = ttest_ind(clicks_control, clicks_experiment)

ex9_ttest_pvalue_clicks = round(ttest_result_clicks.pvalue, 4)

results["ex9_ttest_pvalue_clicks"] = ex9_ttest_pvalue_clicks

print(f"t-test p-value for Clicks: {ex9_ttest_pvalue_clicks}")

t-test p-value for Clicks: 0.9264


In [17]:
results

{'ex4_avg_oec': 0.0941,
 'ex5_avg_guardrail': 0.0822,
 'ex7_ttest_pvalue': 0.8877,
 'ex9_ttest_pvalue_clicks': 0.9264}

## Estimating the Effect of Experiment

### Exercise 10

Now that we've validated our randomization, our next task is to estimate our treatment effect. First, though, there's an issue with your data you've been able to largely ignore until now, but which you should get a grip on before estimating your treatment effect — can you tell what it is and what you should do about it?

Store the number of observations in your data *after* you've addressed this in `ex10_num_obs` (this is mostly meant as a way to sanity check your answer with autograder).

In [18]:
# dealing with missing values

# filter the combined data to exclude rows with missing values in 'Enrollments' or 'Payments'
filtered_data = combined_data.dropna(subset=["Enrollments", "Payments"])


ex10_num_obs = len(filtered_data)
results["ex10_num_obs"] = ex10_num_obs

print(f"Number of observations after addressing missing values: {ex10_num_obs}")

Number of observations after addressing missing values: 46


In [19]:
results

{'ex4_avg_oec': 0.0941,
 'ex5_avg_guardrail': 0.0822,
 'ex7_ttest_pvalue': 0.8877,
 'ex9_ttest_pvalue_clicks': 0.9264,
 'ex10_num_obs': 46}


### Exercise 11

Now that we've established we have good balance (meaning we think randomization was likely successful), we can evaluate the effects of the experiment. Test whether the OEC and the metric you *don't* want affected have different average values in the control group and treatment group. 

Because we've randomized, this is a consistent estimate of the Average Treatment Effect of Udacity's website change.

Calculate the difference in means in your OEC and guardrail metrics using a simple t-test. Store the resulting effect estimates in `ex11_oec_ate` and `ex11_guard_ate` and p-values in `ex11_oec_pvalue` and `ex11_guard_pvalue`. **Please round all answers to 4 decimal places.** Report your ATE in *percentage points*, where `1` denotes 1 percentage point.


In [34]:
# calculate the OEC for both control and experiment groups
filtered_data["OEC"] = (
    filtered_data["Enrollments"] - filtered_data["Payments"]
) / filtered_data["Clicks"]

# for the guardrail metric, we'll use the Clicks/Pageviews ratio to assess
# user engagement without leading directly to enrollments or payments
filtered_data["guardrail_metric"] = filtered_data["Payments"] / filtered_data["Clicks"]

# separate control and experiment groups for both metrics
control_oec = filtered_data[filtered_data["Group"] == "Control"]["OEC"]
experiment_oec = filtered_data[filtered_data["Group"] == "Experiment"]["OEC"]
control_guardrail = filtered_data[filtered_data["Group"] == "Control"][
    "guardrail_metric"
]
experiment_guardrail = filtered_data[filtered_data["Group"] == "Experiment"][
    "guardrail_metric"
]

# t-tests
ttest_result_oec = ttest_ind(experiment_oec, control_oec)
ttest_result_guardrail = ttest_ind(experiment_guardrail, control_guardrail)

# calculate ATE (Average Treatment Effect) for OEC and guardrail metrics
# and convert to percentage points
ex11_oec_ate = (control_oec.mean() - experiment_oec.mean()) * 100
ex11_guard_ate = (control_guardrail.mean() - experiment_guardrail.mean()) * 100


ex11_oec_ate = round(ex11_oec_ate, 4)
results["ex11_oec_ate"] = ex11_oec_ate

ex11_guard_ate = round(ex11_guard_ate, 4)
results["ex11_guard_ate"] = ex11_guard_ate

ex11_oec_pvalue = round(ttest_result_oec.pvalue, 4)
results["ex11_oec_pvalue"] = ex11_oec_pvalue

ex11_guard_pvalue = round(ttest_result_guardrail.pvalue, 4)
results["ex11_guard_pvalue"] = ex11_guard_pvalue


print(
    "The Average Treatment Effect (ATE) on the OEC, expressed in "
    f"percentage points is: {results['ex11_oec_ate']}"
)

print(
    f"The ATE on the guardrail metric, in percentage points, is: {results['ex11_guard_ate']}"
)

print(
    "The p-value for the difference in OEC between groups is: "
    f"{results['ex11_oec_pvalue']}"
)
print(
    f"The p-value for the guardrail metric difference is: {results['ex11_guard_pvalue']}"
)

The Average Treatment Effect (ATE) on the OEC, expressed in percentage points is: 1.5888
The ATE on the guardrail metric, in percentage points, is: 0.4897
The p-value for the difference in OEC between groups is: 0.1319
The p-value for the guardrail metric difference is: 0.5928


### Exercise 12

Do you feel that Udacity achieved their goal? Did their intervention cause them any problems? If they asked you "What would happen if we rolled this out to everyone?" what would you say?

As you answer this question, a small additional question: up until this point you've (presumably) been reporting the default p-values from the tools you are using. These, as you may recall from stats 101, are two-tailed p-values. Do those seem appropriate for your OEC?

>The results from the experiment indicate that while the intervention potentially improved the payment rate per click (a positive sign), it did not significantly enhance the overall evaluation criterion (OEC), which was Udacity's main goal. The good news is, there were no significant negative impacts on user engagement (guardrail metrics), suggesting that the changes didn't harm user experience. Despite the promising increase in payment rate, the lack of a significant improvement in the OEC suggests caution before a full rollout. The experiment's outcomes reveal mixed results: it's beneficial in one aspect but falls short in the primary objective. If Udacity were to ask about rolling this out to everyone, the advice would be to proceed with caution, possibly consider further optimizations or investigations to understand the discrepancy between the improved payment rate and the unchanged OEC. Essentially, the intervention shows potential, but further work is needed to fully achieve Udacity's goals without unintended consequences.

### Exercise 13

One of the magic things about experiments is that all you have to do is compare averages to get an average treatment effect. However, you *can* do other things to try and increase the statistical power of your experiments, like add controls in a linear regression model. 

As you likely know, a bivariate regression is exactly equivalent to a t-test, so let's start by re-estimating the effect of treatment on your OEC using a linear regression. Can you replicate the results from your t-test? They shouldn't just be close—they should be numerically equivalent (i.e. exactly the same to the limits of floating point number precision). 

In [36]:
import statsmodels.api as sm

# we need to add a binary variable to represent the treatment group in the regression model


# create a binary variable for the treatment group where experiment group is 1, control group is 0
filtered_data["treatment"] = filtered_data["Group"].apply(
    lambda x: 1 if x == "Experiment" else 0
)

# define the dependent variable (OEC) and independent variable (treatment group)
Y = filtered_data["OEC"]
X = filtered_data[["treatment"]]
X = sm.add_constant(X)  # Adds a constant term to the predictor

# fit the linear regression model
model = sm.OLS(Y, X).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    OEC   R-squared:                       0.051
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     2.356
Date:                Mon, 18 Mar 2024   Prob (F-statistic):              0.132
Time:                        22:31:02   Log-Likelihood:                 89.832
No. Observations:                  46   AIC:                            -175.7
Df Residuals:                      44   BIC:                            -172.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1021      0.007     13.948      0.000       0.087       0.117
treatment     -0.0159      0.010     -1.535      0.132      -0.037       0.005
==============================================================================
Omnibus:                       14.160   Durbin-Watson:                   1.908
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               15.205
Skew:                           1.227   Prob(JB):                     0.000499
Kurtosis:                       4.383   Cond. No.                         2.62
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Exercise 14

Now add indicator variables for the date of each observation. Do the standard errors on your `treatment` variable change? If so, in what direction?

Store your new standard error in `ex14_se_treatment`. Round your answer to 4 decimal places.

You should have found that your standard errors decreased by about 30\%—this is why, although just comparing means *works*, if you have additional variables adding them to your analysis can be helpful (all the usual rules for model specification apply — for example, you still want to be careful about overfitting, which one could argue is maybe part of what's happening here). 

In many other cases, the effect of adding controls is likely to be larger — the date indicators we added to our data are perfectly balanced between treatment and control, so we aren't adding a lot of data to the model by adding them as variables. They're accounting for some day-to-day variation (presumably in the types of people coming to the site), but they aren't controlling for any residual baseline differences the way a control like "gender" or "age" might (since those kind of individual-level attributes will never be perfectly balanced across treatment and control). 

In [37]:
# convert Date into dummy variables
date_dummies = pd.get_dummies(filtered_data["Date"])

# combine the treatment variable and date dummies
X_with_date = sm.add_constant(
    pd.concat([filtered_data["treatment"], date_dummies], axis=1)
)

# define the dependent variable
Y = filtered_data["OEC"]
X_with_date = X_with_date.astype(int)


# fit the linear regression model with date dummies included
model_with_date = sm.OLS(Y, X_with_date).fit()

ex14_se_treatment = round(model_with_date.bse["treatment"], 4)
results["ex14_se_treatment"] = ex14_se_treatment

print(
    "The new standard error for the treatment variable, after adding "
    f"date indicators, is: {ex14_se_treatment}"
)

The new standard error for the treatment variable, after adding date indicators, is: 0.0066


### Exercise 15

Does this result have any impact on the recommendations you would offer Udacity?

>The reduction in the standard error to 0.0066 after adding date indicators significantly enhances the precision of our estimate of the treatment's effect on the OEC. This improvement means we can trust the results more, giving Udacity a clearer picture of how their website changes are truly performing. If the treatment effect on the OEC remains not statistically significant even with this enhanced precision, it suggests that the website changes may not have the intended positive impact. However, if this finer analysis reveals a significant effect, it signals a more definitive influence of the changes, whether positive or negative. Therefore, with this increased accuracy, Udacity should re-evaluate the statistical and practical significance of the experiment's outcomes. This sharper analysis could justify moving forward with the website changes if the evidence supports it, or it might caution against a full rollout if the desired improvements in user behavior are not evident. In essence, this result provides Udacity with a more reliable foundation for their decision-making regarding the experiment.

In [40]:
results

{'ex4_avg_oec': 0.0941,
 'ex5_avg_guardrail': 0.1158,
 'ex7_ttest_pvalue': 0.8877,
 'ex9_ttest_pvalue_clicks': 0.9264,
 'ex10_num_obs': 46,
 'ex11_oec_ate': 1.5888,
 'ex11_guard_ate': 0.4897,
 'ex11_oec_pvalue': 0.1319,
 'ex11_guard_pvalue': 0.5928,
 'ex14_se_treatment': 0.0066}

In [38]:
assert set(results.keys()) == {
    "ex4_avg_oec",
    "ex5_avg_guardrail",
    "ex7_ttest_pvalue",
    "ex9_ttest_pvalue_clicks",
    "ex10_num_obs",
    "ex11_guard_ate",
    "ex11_guard_pvalue",
    "ex11_oec_ate",
    "ex11_oec_pvalue",
    "ex14_se_treatment",
}